In [1]:
# Force reinstall all langchain packages to the latest matching versions
#%pip install -U --force-reinstall langchain langchain-community langchain-core langchain-google-genai valyu prophet yfinance matplotlib pandas

In [ ]:
import os
import operator
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from typing import Annotated, Literal, TypedDict, List
from prophet import Prophet


# --- LIBRARIES ---
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from langgraph.prebuilt import create_react_agent 
from pydantic import BaseModel, Field
from valyu import Valyu 
from langchain.messages import SystemMessage, HumanMessage
from langchain.chat_models import init_chat_model





model1 = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0,  
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

model = init_chat_model("gpt-4.1")


/Users/burakalicankilinc/.pyenv/versions/3.12.3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
from typing import Annotated, Literal, TypedDict
import operator


class AgentInput(TypedDict):
    """Simple input state for each subagent."""
    query: str


class AgentOutput(TypedDict):
    """Output from each subagent."""
    source: str
    result: str


class Classification(TypedDict):
    """A single routing decision: which agent to call with what query."""
    source: Literal["quant", "research"]
    query: str


class RouterState(TypedDict):
    query: str
    classifications: list[Classification]
    results: Annotated[list[AgentOutput], operator.add]  
    final_answer: str

In [4]:


@tool 
def brownianModel(TICKER: str):
    """ 
    Uses geometric brownian motion and monte carlo method to predict stock prices 30 days ahead.
    Returns technical parameters (Drift, Volatility) and a 90% Confidence Interval.
    """
    try:
        # main variables
        scen_size = 1000 
        HISTORICAL_YEARS = 2
        PREDICTION_DAYS = 30
        stock_name = TICKER

        end_date = pd.Timestamp.today().normalize()
        start_date = end_date - pd.DateOffset(years=HISTORICAL_YEARS)
        pred_end_date = end_date + pd.tseries.offsets.BDay(PREDICTION_DAYS)

        # Download and prepare data
        prices = yf.download(tickers=stock_name, start=start_date, end=pred_end_date, progress=False)
        
        if prices.empty:
            return f"Error: No data found for {stock_name}"

        # Handle yfinance MultiIndex
        if isinstance(prices.columns, pd.MultiIndex):
            prices = prices['Close']
            if isinstance(prices, pd.DataFrame) and stock_name in prices.columns:
                 prices = prices[stock_name]
        elif 'Close' in prices.columns:
            prices = prices['Close']
        
        if isinstance(prices, pd.DataFrame):
             prices = prices.iloc[:, 0]

        future_dates = pd.bdate_range(start=pd.to_datetime(end_date) + pd.Timedelta(days=1),
                        end=pd.to_datetime(pred_end_date))

        train_set = prices.loc[:end_date]
        
        if len(train_set) < 2:
            return "Error: Not enough historical data for prediction."

        daily_returns = ((train_set / train_set.shift(1)) - 1).dropna()
        So = train_set.iloc[-1]
        
        # --- ENRICHED METADATA CALCULATION ---
        mu = np.mean(daily_returns)
        sigma = np.std(daily_returns)
        
        # Annualize for reporting
        annual_drift = mu * 252
        annual_vol = sigma * np.sqrt(252)
        
        if np.isnan(sigma) or sigma == 0:
            return "Error: Volatility calculation failed."

        # Simulation
        T_days = len(future_dates)
        N = T_days
        t = np.arange(1, N + 1)

        # Vectorized Monte Carlo
        b = {str(scen): np.random.normal(0, 1, N) for scen in range(1, scen_size + 1)}
        W = {str(scen): b[str(scen)].cumsum() for scen in range(1, scen_size + 1)}

        drift = (mu - 0.5 * sigma ** 2) * t
        diffusion = {str(scen): sigma * W[str(scen)] for scen in range(1, scen_size + 1)}

        S = np.array([So * np.exp(drift + diffusion[str(scen)]) for scen in range(1, scen_size + 1)])
        
        # Stats: Mean, 5th percentile (Bear), 95th percentile (Bull)
        S_pred = np.mean(S, axis=0)
        S_low = np.percentile(S, 5, axis=0)
        S_high = np.percentile(S, 95, axis=0)

        final_df = pd.DataFrame({
            'mean': S_pred,
            'low': S_low,
            'high': S_high
        }, index=future_dates[:len(S_pred)])

        # Create detailed output string
        rows = []
        for date, row in final_df.iterrows():
            rows.append(f"{date.date()}: Mean ${row['mean']:.2f} (Range: ${row['low']:.2f} - ${row['high']:.2f})")
            
        daily_data = '\n'.join(rows)

        # RETURN THE "HOW" DATA
        return (f"Brownian Motion Analysis for {stock_name}:\n"
                f"--- TECHNICAL PARAMETERS ---\n"
                f"Historical Data Points: {len(train_set)}\n"
                f"Annualized Volatility (Sigma): {annual_vol:.2%}\n"
                f"Annualized Drift (Mu): {annual_drift:.2%}\n"
                f"Simulations Run: {scen_size}\n"
                f"Confidence Interval: 90% (5th to 95th percentile)\n"
                f"----------------------------\n"
                f"Forecast Data:\n{daily_data}")

    except Exception as e:
        return f"Brownian Model failed: {str(e)}"

@tool
def mlModel(ticker: str):
    """ 
    Uses Facebook Prophet to predict stock prices 30 days ahead.
    """
    try:
        data = yf.Ticker(ticker).history(period="2y")
        if data.empty:
            return "Error: No data found"
        
        df = data.reset_index()
        df['ds'] = df['Date'].dt.tz_localize(None)
        df['y'] = df['Close']

        m = Prophet(daily_seasonality=True)
        m.fit(df)

        future = m.make_future_dataframe(periods=30)
        forecast = m.predict(future)

        # Plot logic (omitted for brevity, keep your original plot logic here if needed)
        graph_filename = f"{ticker}_forecast.png"
        # ... (keep plot saving code if you want) ...

        future_data = forecast.tail(30)
        latest_pred = forecast.iloc[-1]['yhat']
        current_price = df.iloc[-1]['y']
        trend = "UP" if latest_pred > current_price else "DOWN"
        
        # ENRICHED OUTPUT
        return (f"Machine Learning (Prophet) Analysis for {ticker}\n"
                f"--- MODEL DETAILS ---\n"
                f"Algorithm: Additive Regression Model (Prophet)\n"
                f"Training Data: {len(df)} days\n"
                f"Seasonality: Daily/Weekly enabled\n"
                f"Trend Detected: {trend}\n"
                f"---------------------\n"
                f"Daily Price Targets (Next 30 Days):\n"
                f"{future_data[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(30).to_string()}")
    
    except Exception as e:
        return f"Prediction failed: {e}"


@tool
def valyu_search_tool(query: str):
    """
    Searches the web using Valyu API to get specific market news and context.
    """
    try:
        # Using the .answer method from your provided code
        response = Valyu.answer(query)
        
        # Extract content safely
        if hasattr(response, 'contents'):
            return response.contents
        elif isinstance(response, dict) and 'contents' in response:
            return response['contents']
        else:
            return str(response)
            
    except Exception as e:
        return f"Search failed: {e}"
    







In [5]:
trend_prompt = (
    "You are a Quantitative Analyst. Use the provided ML and Statistical tools to analyze the stock ticker provided. "
    "ONLY ENTER THE ABBREVIATION OF THE STOCK TO THE TOOLS. "
    "Your report must be detailed and data-heavy. You MUST include:\n"
    "1. The exact current price of the stock.\n"
    "2. The specific daily price targets for the next 30 days from the models.\n"
    "3. The median prediction and confidence intervals from the Brownian motion model.\n"
    "4. A clear statement of the trend direction (UP/DOWN/FLAT) based on the math.\n"
    "5. If a tool fails, explicitly state why (e.g., 'Not enough data')."
)
trend_agent = create_agent(model, system_prompt=SystemMessage(content=[{"type": "text", "text": trend_prompt}, {"type": "text", "text": "stock markets"}], ), tools=[mlModel, brownianModel])

noise_prompt = (
    "You are a Market Researcher. Use the search tool to find recent news, sentiment, and macro factors affecting the stock. "
    "Do not just summarize; provide a detailed list of findings. You MUST include:\n"
    "1. Specific headlines, dates, and sources of the news you found.\n"
    "2. Direct quotes or key statistics from the search results.\n"
    "3. Any upcoming events (earnings dates, product launches).\n"
    "4. The overall market sentiment supported by specific evidence."
)
noise_agent = create_agent(model, [valyu_search_tool], system_prompt=SystemMessage(content=[{"type": "text", "text": noise_prompt}, {"type": "text", "text": "stock markets"}], ))


In [6]:
#result = trend_agent.invoke({"messages": [HumanMessage("analyze AMZN stock")]})

#ai_message = result["messages"][-1]


In [7]:
from pydantic import BaseModel, Field
from langgraph.types import Send
from langgraph.graph import StateGraph, START, END

class ClassificationResult(BaseModel):  
    """Result of classifying a user query into agent-specific sub-questions."""
    classifications: list[Classification] = Field(
        description="List of agents to invoke with their targeted sub-questions"
    )

def classify_query(state: RouterState) -> dict:
    """Classify query and determine which agents to invoke."""
    structured_llm = model.with_structured_output(ClassificationResult)  

    result = structured_llm.invoke([
        {
            "role": "system",
            "content": """ Analyze this query and find the stock that the user wants to make predictions on. Determine the abbreviation of the stock. 
            pass the abbreviation to your tools. You have two tools:
            1- Quant: gets information on the stock and runs a brownian model and a ML model. These model predict future state of the stock.
            2- research: researches on the internet and finds relevant data regarding the stock

            only pass the abbreviation of the stock, nothing else. 

            example for "make predictions on Amazon stock":
            AMZN
            

            """
        },
        {"role": "user", "content": state["query"]}
    ])

    return {"classifications": result.classifications}

def route_to_agents(state: RouterState) -> list[Send]:
    """Fan out to agents based on classifications."""
    return [
        Send(c["source"], {"query": c["query"]})  
        for c in state["classifications"]
    ]


def run_trend_agent(state: RouterState):
    """Invokes the Quant Agent"""
    print("Executing Trend Agent")
    response = trend_agent.invoke({"messages": [{"role": "user", "content": state["query"]}]})
    
    return {"results": [{"source": "quant", "result": response["messages"][-1].content}]}

def run_noise_agent(state: RouterState):
    """Invokes the Research Agent"""
    print("Executing Noise Agent")
    response = noise_agent.invoke({"messages": [{"role": "user", "content": state["query"]}]})
    
    return {"results": [{"source": "research", "result": response["messages"][-1].content}]}

def synthesize_results(state: RouterState) -> dict:
    """Combine results from all agents into a comprehensive report."""
    if not state["results"]:
        return {"final_answer": "No results found from any knowledge source."}

    formatted = [
        f"--- REPORT FROM {r['source'].upper()} DEPARTMENT ---\n{r['result']}\n------------------------------------------------"
        for r in state["results"]
    ]

    # UPDATE: Prompt now demands technical explanation
    synthesis_prompt = f"""You are a Senior Investment Analyst compiling a comprehensive Due Diligence Report.
    The user asked: "{state['query']}"

    Your goal is to provide a "White Box" analysis—explaining NOT just the prediction, but HOW the math worked.

    STRICTLY FOLLOW THIS REPORT STRUCTURE:

    1. **Executive Summary**
       - A high-level verdict (Buy/Sell/Hold/Wait).

    2. **Methodology & Technical Deep-Dive (CRITICAL SECTION)**
       - Explain the logic behind the models.
       - **Brownian Motion:** Explicitly state the "Annualized Volatility" and "Drift" percentages found in the tool output. Explain what they mean (e.g., "The model calculated a historical volatility of 25%, implying...").
       - **ML Model:** Mention the training data size and algorithm.
    
    3. **Quantitative Analysis (The Numbers)**
       - Detail the price targets.
       - **Range:** Quote the 90% Confidence Interval (Low/High) provided by the Brownian model.

    4. **Market Context (The News)**
       - Summarize the news headlines and sentiment.
       - CITE SOURCES.

    5. **Risk Factors & Conclusion**
       - specific risks (e.g., "High volatility of X% increases downside risk").

    Do not shorten the content. USE THE TECHNICAL PARAMETERS (Sigma, Mu, CI) PROVIDED IN THE TEXT."""

    synthesis_response = model.invoke([
        {
            "role": "system",
            "content": synthesis_prompt
        },
        {"role": "user", "content": "\n\n".join(formatted)}
    ])

    return {"final_answer": synthesis_response.content}

workflow = (
    StateGraph(RouterState)
    .add_node("classify", classify_query)
    .add_node("quant", run_trend_agent)
    .add_node("research", run_noise_agent)
    .add_node("synthesize", synthesize_results)
    .add_edge(START, "classify")
    .add_conditional_edges("classify", route_to_agents, ["quant", "research"])
    .add_edge("quant", "synthesize")
    .add_edge("research", "synthesize")
    .add_edge("synthesize", END)
    .compile()
)



result = workflow.invoke({
    "query": "can you make predictions on Amazon stock?"
})

print("Original query:", result["query"])
print("\nClassifications:")
for c in result["classifications"]:
    print(f"  {c['source']}: {c['query']}")
print("\n" + "=" * 60 + "\n")
print("Final Answer:")
print(result["final_answer"])








Executing Trend Agent


11:44:29 - cmdstanpy - INFO - Chain [1] start processing
11:44:30 - cmdstanpy - INFO - Chain [1] done processing


Original query: can you make predictions on Amazon stock?

Classifications:
  quant: AMZN


Final Answer:
Amazon.com, Inc. (AMZN)  
Comprehensive Predictive Analysis – February–March 2026

---

## 1. Executive Summary

**Recommendation:** **Wait / Hold**  
Currently, quantitative forecasts for Amazon (AMZN) display conflicting signals: the predominant machine learning model (Prophet) projects a steady **decline** in share price over the next month, while the stochastic Brownian motion model signals a **flat but highly volatile** trajectory. Given this technical incongruence, the prudent action is to **wait** for further confirmation before adding or expanding positions. Downside volatility risk is material.

---

## 2. Methodology & Technical Deep-Dive

### A. Machine Learning Model (Prophet)

- **Logic:** Prophet is an additive, decomposable time series model (trend + seasonality + holidays + noise), trained here on multiple years of Amazon daily closing prices (training data: >1,000 